In [1]:
data_file_path=r'D:\NLP\github\中文新闻\cnews.train.txt'
with open(data_file_path,'r',encoding='utf-8') as f:
    lines=f.readlines()

titles=[]
texts=[]
for news in lines[:10000]:
    news=news[3:]
    if('新浪体育讯' in news):
        title=news.split('新浪体育讯')[0]
        text=''.join(news.split('新浪体育讯')[1:])
        titles.append(title.strip())
        texts.append(text.strip())
print(len(titles))
print(len(texts))

4246
4246


In [2]:
print(titles[1000])
print(texts[1000])

NBA最性感女球迷裸照 手拉胸衣露薄纱浑圆半球(图)
尽管拜伦-戴维斯至今还处在伤病当中，但他距离复出的日子已经不远了。据洛杉矶媒体的消息，戴维斯现在已经恢复了和全队的合练，并且将在下周重新回到赛场。到时候，人们又能看到场边那个性感女神一般的女人为他在现场加油助威了。戴维斯和杰西卡还有他的丈夫都是非常要好的朋友，杰西卡一有空都会去给戴维斯和他的球队捧场，她也很快成为了戴维斯的钟情粉丝。因此，她也是NBA中除了帕克的妻子伊娃之外，知名度最高的女球迷。杰西卡-阿尔芭年轻时就拥有很高的表演天赋，她13岁开始拍片，2005年通过主演《罪恶之城》、《神奇四侠》达到事业顶峰。虽然大多以青春靓丽的“花瓶”形象出现，但她的美丽身影通常是影片中最迷人的风景。连续三年入选了《男人帮》性感女星评选的前十名，是美国“最性感的女星”之一。有球迷甚至把她称作是甜美的巧克力美人，她1981年4月28日出生于加利福尼亚的波莫纳市，拥有5国混血血统。妈妈拥有加拿大、法国、和丹麦人的混血，爸爸是墨西哥和美国的混血儿。这就解释了杰西卡棕色的肌肤和漂亮的深棕色眼睛。金牛座的女人天生就是一个精力充沛，生活欲望强烈且楚楚动人的女生。美国的著名演员杰西卡-阿尔芭就是这样的，她很会按照女人的特点无忧无虑地生活。她魅力无限，渴望经历爱情生活的全过程：恋爱、结婚、家庭、孩子和美味佳肴对她来说都是生活中不可错过的事情。令人一直感到意外的是身在好莱坞这个大染缸，杰西卡依然能坚守住自己的那片处女地。她曾连续三年被评为“全球最性感的女人”前五名，可是她享受性爱却拒绝裸戏。她甚至把《花花公子》告上法庭，原因是《花花公子》在没有征求她允许的情况下擅自“盗用”了她的比基尼剧照作了杂志封面。虽然一直在娱乐圈里留下了圣洁的名声，但最近网站上曝光的一组露点照片却让杰西卡-阿尔芭陷入了窘境。在美国网站《egotastic》曝光的这组照片中，有多张杰西卡-阿尔芭大着肚子的露点照片，这让人们完全改变了以往她在人们心中留下的印象。当初和花花公子闹上公堂的时候，杰西卡曾经对媒体说：“你可能不会相信，我拒绝裸戏！在众人面前不穿衣服我会不知所措！登上《花花公子》封面这会给很多人误解，以为里面会有我的裸照。”可现在，她的半裸照片却清晰的被挂在了网站上。只是不知道戴维斯看了这些照片之后会作何感想。(乳娃娃)


In [3]:
titles_lens=[]
texts_lens=[]
for line in titles:
    titles_lens.append(len(line))
for line in texts:
    texts_lens.append(len(line))
titles_lens.sort()
print('title_len_avg:%f'%(sum(titles_lens)/len(titles)))
print('title_len_middle:%f'%(titles_lens[int(len(titles)/2)]))
print('title_len_min:%f'%(titles_lens[0]))
print('title_len_max:%f'%(titles_lens[len(titles)-1]))
texts_lens.sort()
print('text_len_avg:%f'%(sum(texts_lens)/len(texts)))
print('text_len_middle:%f'%(texts_lens[int(len(texts)/2)]))
print('text_len_min:%f'%(texts_lens[0]))
print('text_len_max:%f'%(texts_lens[len(texts)-1]))


title_len_avg:23.757654
title_len_middle:23.000000
title_len_min:13.000000
title_len_max:30.000000
text_len_avg:838.758832
text_len_middle:813.000000
text_len_min:48.000000
text_len_max:12395.000000


In [4]:
import jieba
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

Using TensorFlow backend.


In [5]:
MAX_WORDS=10000

#将句子分词并用空格隔开
inputTextList=[' '.join([w for w in jieba.cut(text)]) for text in texts]
targetTextList=[' '.join([w for w in jieba.cut(text)]) for text in titles]

#-4的原因是后面会加上4个特殊字符串，值从1开始，故词典里只设置9999个词
tokenizer=Tokenizer(num_words=MAX_WORDS-5)
tokenizer.fit_on_texts(texts=inputTextList+targetTextList)

word_index=tokenizer.word_index

# 增加特殊编码
SPECIAL_CODES = ['<PAD>', '<EOS>', '<UNK>', '<GO>']

for i,w in enumerate(SPECIAL_CODES):
    word_index[w]=MAX_WORDS-i-1

re_word_index = dict([(i, t) for t, i in word_index.items()])

with open('AutoDigestGeneration_Dict.txt','w',encoding='utf-8') as f:
    f.write(str(word_index))

#将文本映射为数字序列
input_sequences=tokenizer.texts_to_sequences(texts=inputTextList)
target_sequences=tokenizer.texts_to_sequences(texts=targetTextList)

print(len(word_index))
print(len(re_word_index))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1.M6S\AppData\Local\Temp\jieba.cache
Loading model cost 0.761 seconds.
Prefix dict has been built succesfully.


46705
46701


In [6]:
print(input_sequences[1000])

for n in input_sequences[1000]:
    print(re_word_index[n],end='')

[372, 2461, 735, 1207, 38, 2543, 398, 1555, 1, 27, 6, 852, 290, 2, 2085, 51, 4916, 5, 3, 317, 387, 444, 2, 133, 1, 735, 106, 51, 670, 5, 11, 371, 2, 4303, 1, 496, 33, 4, 3936, 516, 397, 1107, 3, 4316, 1, 710, 94, 67, 348, 1199, 642, 4477, 7280, 816, 2, 3703, 104, 6, 4, 588, 3166, 7360, 5, 3, 735, 11, 8336, 135, 6, 2, 9755, 20, 7, 132, 93, 107, 2, 1797, 1, 8336, 251, 20, 61, 178, 118, 735, 11, 6, 2, 19, 8776, 1, 1138, 10, 733, 165, 5, 735, 2, 3695, 3, 501, 1, 1138, 10, 7, 128, 14, 505, 680, 2, 3085, 2372, 922, 1, 441, 2, 3, 8336, 7739, 538, 64, 24, 519, 43, 556, 2, 858, 1090, 1, 1138, 141, 531, 112, 1, 1825, 102, 652, 7251, 374, 578, 1965, 383, 21, 374, 383, 426, 6678, 3, 145, 4118, 60, 5546, 2, 12, 13, 3851, 302, 1, 27, 1138, 2, 3693, 2672, 7, 14, 152, 2, 9424, 3, 167, 1721, 1349, 5, 374, 383, 7763, 4252, 2, 1, 7, 609, 12, 152, 4477, 2, 13, 535, 3, 18, 154, 335, 164, 1138, 7436, 7, 2, 1, 1138, 8031, 102, 53, 37, 410, 7466, 2, 1, 519, 56, 3, 4338, 519, 5388, 21, 4191, 21, 11, 58, 2, 1, 

In [7]:
titles_lens=[]
texts_lens=[]
for line in input_sequences:
    titles_lens.append(len(line))
for line in target_sequences:
    texts_lens.append(len(line))
titles_lens.sort()
print('title_len_avg:%f'%(sum(titles_lens)/len(titles)))
print('title_len_middle:%f'%(titles_lens[int(len(titles)/2)]))
print('title_len_min:%f'%(titles_lens[0]))
print('title_len_max:%f'%(titles_lens[len(titles)-1]))
texts_lens.sort()
print('text_len_avg:%f'%(sum(texts_lens)/len(texts)))
print('text_len_middle:%f'%(texts_lens[int(len(texts)/2)]))
print('text_len_min:%f'%(texts_lens[0]))
print('text_len_max:%f'%(texts_lens[len(texts)-1]))

title_len_avg:478.837965
title_len_middle:465.000000
title_len_min:26.000000
title_len_max:6457.000000
text_len_avg:10.961611
text_len_middle:11.000000
text_len_min:3.000000
text_len_max:19.000000


In [8]:
#因此确定
Tx=500
Ty=20

In [9]:
#padding数据
import tqdm

input_arr=[]
target_arr=[]

for line in tqdm.tqdm(input_sequences):
    slen=len(line)
    if(slen<Tx):
        newline=line+[word_index['<PAD>']]*(Tx-slen)
        input_arr.append(newline)
    else:
        input_arr.append(line[:Tx])


for line in tqdm.tqdm(target_sequences):
    slen=len(line)
    if(slen<Ty):
        line.append(word_index['<EOS>'])
        newline=line+[word_index['<PAD>']]*(Ty-slen-1)
        target_arr.append(newline)
    else:
        target_arr.append(line[:Ty])

input_arr=np.array(input_arr)
target_arr=np.array(target_arr)

print(input_arr.shape)
print(target_arr.shape)

100%|██████████████████████████████████████████████████████████████████████████| 4246/4246 [00:00<00:00, 428060.16it/s]


(4246, 500)
(4246, 20)


In [10]:
print(input_arr[1000])
print(target_arr[1000])

[ 372 2461  735 1207   38 2543  398 1555    1   27    6  852  290    2
 2085   51 4916    5    3  317  387  444    2  133    1  735  106   51
  670    5   11  371    2 4303    1  496   33    4 3936  516  397 1107
    3 4316    1  710   94   67  348 1199  642 4477 7280  816    2 3703
  104    6    4  588 3166 7360    5    3  735   11 8336  135    6    2
 9755   20    7  132   93  107    2 1797    1 8336  251   20   61  178
  118  735   11    6    2   19 8776    1 1138   10  733  165    5  735
    2 3695    3  501    1 1138   10    7  128   14  505  680    2 3085
 2372  922    1  441    2    3 8336 7739  538   64   24  519   43  556
    2  858 1090    1 1138  141  531  112    1 1825  102  652 7251  374
  578 1965  383   21  374  383  426 6678    3  145 4118   60 5546    2
   12   13 3851  302    1   27 1138    2 3693 2672    7   14  152    2
 9424    3  167 1721 1349    5  374  383 7763 4252    2    1    7  609
   12  152 4477    2   13  535    3   18  154  335  164 1138 7436    7
    2 

In [11]:
# 自定义softmax函数
def softmax(x, axis=1):
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')
        

In [12]:
from keras.layers import *

# 定义全局网络层对象
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor_tanh = Dense(32, activation = "tanh")
densor_relu = Dense(1, activation = "relu")
densor_con = Dense(256, activation = "relu")
activator = Activation(softmax, name='attention_weights')
dotor = Dot(axes = 1)

In [13]:
def one_step_attention(a, s_prev):
    # 将s_prev复制Tx次
    s_prev = repeator(s_prev)
    # 拼接BiRNN隐层状态与s_prev
    concat = concatenator([a, s_prev])
    # 计算energies
    e = densor_tanh(concat)
    energies = densor_relu(e)
    # 计算weights
    alphas = activator(energies)
    # 加权得到Context Vector
    context = dotor([alphas, a])
    
    return context

In [14]:
# 加载预训练好的glove词向量
with open(r'D:\NLP\wordvector\sgns.zhihu.word\sgns.zhihu.word', 'r',encoding='utf-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
        line = line.strip().split()
        curr_word = line[0]
        words.add(curr_word)
        word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

In [15]:
def pretrained_embedding_layer(word_to_vec_map, word_index):
    vocab_len = len(word_index) + 1        # Keras Embedding的API要求+1
    emb_dim = 300
    # 初始化embedding矩阵
    emb_matrix = np.zeros((vocab_len, emb_dim))
    # 用词向量填充embedding矩阵
    for word, index in word_index.items():
        word_vector = word_to_vec_map.get(word, np.zeros(emb_dim))
        emb_matrix[index, :] = word_vector
    # 定义Embedding层，并指定不需要训练该层的权重
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    # build
    embedding_layer.build((None,))
    # set weights
    embedding_layer.set_weights([emb_matrix])
    return embedding_layer

In [16]:
# 获取Embedding layer
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_index)

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
from keras.utils import to_categorical

def train_gen(X,Y,Ty,n_s,batch_size=64):
    xlen=X.shape[0]
    permutation = np.random.permutation(xlen)
    x = X[permutation]
    y = Y[permutation]
    num_batches = int(xlen/batch_size)
    
    while 1:
        for i in range(num_batches):
            x_batch=x[i*batch_size:(i+1)*batch_size]
            y_batch=y[i*batch_size:(i+1)*batch_size]
            s0=np.zeros((batch_size,n_s))
            c0=np.zeros((batch_size,n_s))
            out0=np.zeros((batch_size,MAX_WORDS))#加一的原因是字典中的值从1开始
            outputs=np.zeros((batch_size,Ty,MAX_WORDS))
            for i,line in enumerate(y_batch):
                for j,index in enumerate(line):
                    outputs[i,j,index]=1
                    
            #outputs=np.array(list(map(lambda x: to_categorical(x, num_classes=MAX_WORDS), y_batch)))
            outputs = list(outputs.swapaxes(0,1))
            yield [x_batch,s0,c0,out0],outputs

In [18]:
n_a = 128 # The hidden size of Bi-LSTM
n_s = 128 # The hidden size of LSTM in Decoder
decoder_LSTM_cell = LSTM(n_s, return_state=True)
output_layer = Dense(MAX_WORDS, activation=softmax)

In [19]:
# 定义网络层对象（用在model函数中）
reshapor = Reshape((1, MAX_WORDS))
concator = Concatenate(axis=-1)

In [20]:
def model(Tx, Ty, n_encoder, n_decoder):
    """
    构造模型
    @param Tx: 输入序列的长度
    @param Ty: 输出序列的长度
    @param n_encoder: Encoder端Bi-LSTM隐层结点数
    @param n_decoder: Decoder端LSTM隐层结点数
    """
    
    # 定义输入层
    X = Input(shape=(Tx,))
    # Embedding层
    embed = embedding_layer(X)
    
    # 定义Bi-LSTM
    a = Bidirectional(LSTM(n_decoder, return_sequences=True))(embed)
    
    # Decoder端LSTM的初始状态
    s0 = Input(shape=(n_decoder,), name='s0')
    c0 = Input(shape=(n_decoder,), name='c0')
    
    # Decoder端LSTM的初始输入
    out0 = Input(shape=(MAX_WORDS, ), name='out0')
    out = reshapor(out0)
    
    s = s0
    c = c0
    
    # 模型输出列表，用来存储翻译的结果
    outputs = []
    
    # Decoder端，迭代Ty轮，每轮生成一个翻译结果
    for t in range(Ty):
        # 获取Context Vector
        context = one_step_attention(a, s)
        
        # 将Context Vector与上一轮的翻译结果进行concat
        con_out=densor_con(reshapor(out))
        context = concator([context,con_out])
        s, _, c = decoder_LSTM_cell(context, initial_state=[s, c])
        
        # 将LSTM的输出结果与全连接层链接
        out = output_layer(s)
        
        # 存储输出结果
        outputs.append(out)
    
    model = Model([X, s0, c0, out0], outputs)
    
    return model

In [21]:
model = model(Tx, Ty, n_a, n_s)

In [22]:
from keras.utils import plot_model

plot_model(model,to_file='attentionTest_model.png',show_shapes=True)

In [ ]:
#model.summary()

In [ ]:
from keras.optimizers import Adam
from keras.models import load_model
import keras.backend as K
'''
out = model.compile(optimizer=Adam(lr=0.1, beta_1=0.9, beta_2=0.999, decay=0.001),
                    metrics=['accuracy'],
                    loss='categorical_crossentropy')
'''
out = model.compile(optimizer='rmsprop',
                    metrics=['accuracy'],
                    loss='categorical_crossentropy')

In [ ]:
# 训练模型
xlen=len(input_arr)

model.fit_generator(train_gen(input_arr,target_arr,Ty,n_s,batch_size=8),
                   steps_per_epoch=int(xlen/4),
                   epochs=3)

In [ ]:
model.save('AutoDigestGenerationTest.h5')
model.save_weights('AutoDigestGenerationTest_weights.h5')

In [ ]:
from keras.models import load_model

model=load_model('AutoDigestGenerationTest.h5')

In [ ]:
import jieba

with open('AutoDigestGeneration_Dict.txt','r',encoding='utf-8') as f:
    word_index_str=f.read()
    

    
word_index=eval(word_index_str)
re_word_index = dict([(i, t) for t, i in word_index.items()])
Tx=500
n_decoder=128

def make_prediction(sentence):
    # 将句子分词后转化为数字编码
    input_seq=[]
    for w in jieba.cut(sentence):
        if w in word_index:
            input_seq.append(word_index[w])
            
    if(len(input_seq)<=Tx):
        input_seq = np.array(input_seq + [word_index['PAD']] * (Tx - len(input_seq)))
    else:
        input_seq = np.array(input_seq[:Tx])
    s0=np.zeros((1,n_decoder))
    c0=np.zeros((1,n_decoder))
    out0=np.zeros((1,MAX_WORDS))
    
    print(input_seq)
    
    # 翻译结果
    preds = model.predict([input_seq.reshape(-1,Tx), s0, c0, out0])
    predictions = np.argmax(preds, axis=-1)
    
    print(predictions)
    
    # 转换为单词
    idx = [re_word_index.get(idx[0], "<UNK>") for idx in predictions]
    
    # 返回句子
    return " ".join(idx)

text='''
尽管拜伦-戴维斯至今还处在伤病当中，但他距离复出的日子已经不远了。据洛杉矶媒体的消息，戴维斯现在已经恢复了和全队的合练，并且将在下周重新回到赛场。到时候，人们又能看到场边那个性感女神一般的女人为他在现场加油助威了。戴维斯和杰西卡还有他的丈夫都是非常要好的朋友，杰西卡一有空都会去给戴维斯和他的球队捧场，她也很快成为了戴维斯的钟情粉丝。因此，她也是NBA中除了帕克的妻子伊娃之外，知名度最高的女球迷。杰西卡-阿尔芭年轻时就拥有很高的表演天赋，她13岁开始拍片，2005年通过主演《罪恶之城》、《神奇四侠》达到事业顶峰。虽然大多以青春靓丽的“花瓶”形象出现，但她的美丽身影通常是影片中最迷人的风景。连续三年入选了《男人帮》性感女星评选的前十名，是美国“最性感的女星”之一。有球迷甚至把她称作是甜美的巧克力美人，她1981年4月28日出生于加利福尼亚的波莫纳市，拥有5国混血血统。妈妈拥有加拿大、法国、和丹麦人的混血，爸爸是墨西哥和美国的混血儿。这就解释了杰西卡棕色的肌肤和漂亮的深棕色眼睛。金牛座的女人天生就是一个精力充沛，生活欲望强烈且楚楚动人的女生。美国的著名演员杰西卡-阿尔芭就是这样的，她很会按照女人的特点无忧无虑地生活。她魅力无限，渴望经历爱情生活的全过程：恋爱、结婚、家庭、孩子和美味佳肴对她来说都是生活中不可错过的事情。令人一直感到意外的是身在好莱坞这个大染缸，杰西卡依然能坚守住自己的那片处女地。她曾连续三年被评为“全球最性感的女人”前五名，可是她享受性爱却拒绝裸戏。她甚至把《花花公子》告上法庭，原因是《花花公子》在没有征求她允许的情况下擅自“盗用”了她的比基尼剧照作了杂志封面。虽然一直在娱乐圈里留下了圣洁的名声，但最近网站上曝光的一组露点照片却让杰西卡-阿尔芭陷入了窘境。在美国网站《egotastic》曝光的这组照片中，有多张杰西卡-阿尔芭大着肚子的露点照片，这让人们完全改变了以往她在人们心中留下的印象。当初和花花公子闹上公堂的时候，杰西卡曾经对媒体说：“你可能不会相信，我拒绝裸戏！在众人面前不穿衣服我会不知所措！登上《花花公子》封面这会给很多人误解，以为里面会有我的裸照。”可现在，她的半裸照片却清晰的被挂在了网站上。只是不知道戴维斯看了这些照片之后会作何感想。(乳娃娃)
'''
print(text)
result=make_prediction(text)
print(result)

In [ ]:
print(len(word_index))